In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt

from utils.utils_CNN_v2 import utilsCNN
from Visualization.plot_graph import plot_graph

utilsCNN = utilsCNN()
plots = plot_graph()

# %matplotlib tk
np.random.seed(7)
np.set_printoptions(precision=4)

labelData_dataDir = 'D:/바탕화면/2020_tug_walkcam_dataset/TUG_dataset/arrangedData_LabelSave' ; 
print(" Current workaing path [root]", os.getcwd())  
datasetDir =  labelData_dataDir
os.chdir(datasetDir) ; print(" Current workaing path [dataset] ", os.getcwd())  # -- Dataset 있는 곳으로 경로 변경 

# -------- 1. get csv file list for every trial 
utilsCNN.getDatalist_forEveryTrials(ROOT_DIR = datasetDir, displayList=True)

# # # ------ 2. getRawPelvis from All Trial (draw and save figures)
utilsCNN.getData_forEveryTrial(ROOT_DIR = datasetDir, displayResult = False)  #

utils CNN 객체 생성 -- Initialize
 Current workaing path [root] d:\바탕화면\Conda_WS\tug_pjt\own\azureSkeleton_usingCNN
 Current workaing path [dataset]  D:\바탕화면\2020_tug_walkcam_dataset\TUG_dataset\arrangedData_LabelSave
Folder index: 0 ,    Folder name: 2020_11_03_(result) 
     ----------------------------------------------------------------------         
     Subject Idx: 0,      Subject: bys_tug ,   # fo csvFiles: 9
     ----------------------------------------------------------------------         
     Subject Idx: 1,      Subject: cbd_tug ,   # fo csvFiles: 10
     ----------------------------------------------------------------------         
     Subject Idx: 2,      Subject: cyj_tug ,   # fo csvFiles: 10
Folder index: 1 ,    Folder name: 2020_11_20_v1_(result) 
     ----------------------------------------------------------------------         
     Subject Idx: 3,      Subject: NHJ_60 ,   # fo csvFiles: 9
     ---------------------------------------------------------------------- 

In [2]:
numFeature = 4 # timestamp_sec, pelvis_x, pelvis_y, pelvis_z
numActions = 5  # "sitting": 0, "sit-stand":1, "walking":2, "turning":3, "stand-sit":4
action_label_code = {"sitting": 0, "sit-stand":1, "walking":2, "turning":3, "stand-sit":4} 

train_x , train_y , test_x, test_y= utilsCNN.createDataset_fromCSV(ROOT_DIR=datasetDir, numFeature=numFeature, numActions = numActions, trainSubNum = 13, displayInfo=False)
print("Train Set(x,y): {0} , {1} , Test Set(x,y): {2} , {3}".format(train_x.shape,train_y.shape, test_x.shape, test_y.shape))

Train Set(x,y): (50828, 4) , (50828, 5) , Test Set(x,y): (23438, 4) , (23438, 5)


In [3]:
sliding_window_size = 15 # 30.. 1sec.. 50 Equals to 1 sec for MotionSense Dataset (it is on 50Hz samplig rate)
step_size_of_sliding_window = 3 #10 
print("--> Sectioning Training and Test datasets: shape of each section will be: (",numFeature,"x",sliding_window_size,")")
train_data, act_train_labels, train_mean, train_std = utilsCNN.time_series_2d_to_3d_section(train_x.copy(),train_y.copy(), numActions, sliding_window_size,step_size_of_sliding_window,standardize = True)

test_data, act_test_labels, test_mean, test_std = utilsCNN.time_series_2d_to_3d_section(test_x.copy(), test_y.copy(), 
                                                                                          numActions,
                                                                                          sliding_window_size,
                                                                                          step_size_of_sliding_window,
                                                                                          standardize = True,
                                                                                          mean = train_mean, 
                                                                                          std = train_std)

print("---> Training Section: {0} , Test Sections: {1}".format(train_data.shape, test_data.shape))

--> Sectioning Training and Test datasets: shape of each section will be: ( 4 x 15 )
Training Data has been standardized:
 the mean is =  1210.874425609768  ; and the std is =  451.5064356237748
number of sections:  16938
[-1.6878 -1.6795 -1.6713 -1.659  -1.6508 -1.6426 -1.6344 -1.6262 -1.618
 -1.6098 -1.6016 -1.5934 -1.5852 -1.577  -1.5687]
Test Data has been standardized:
number of sections:  7808
[-1.6878 -1.6798 -1.6672 -1.6593 -1.6511 -1.6429 -1.6347 -1.6265 -1.6177
 -1.61   -1.6016 -1.5936 -1.5852 -1.5772 -1.5649]
---> Training Section: (13298, 4, 15) , Test Sections: (6161, 4, 15)


In [4]:
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import Model 
from tensorflow.keras.layers import Input, Dense, Flatten, Reshape
#from tensorflow.keras.layers import Convolution2D, MaxPooling2D, Dropout
from tensorflow.keras.layers import Conv2D , MaxPooling2D, Dropout

## Here we add an extra dimension to the datasets just to be ready for using with Convolution2D
train_data = np.expand_dims(train_data, axis=3)
test_data = np.expand_dims(test_data, axis=3)
print("---> Training Section: {0} , Test Sections: {1}".format(train_data.shape, test_data.shape))

---> Training Section: (13298, 4, 15, 1) , Test Sections: (6161, 4, 15, 1)


In [5]:
num_train, height, width, channel = train_data.shape  #   (61728, 4, 50, 1) =  (number_of_secs , size_features , sliding_window_size, 1)
metrics = ['acc']

## MTCNN
drop_prob_1 = 0.2 
drop_prob_2 = 0.4 


## Action model 
action_layer_dim = numActions # 4  # dws, ups, wlk, jog # , sit,  std 
action_activation_func = 'softmax'
action_loss_func = 'categorical_crossentropy'


# feature 가 row 에 하나씩 있음 -> kernel, pooling  시 (1, k) 로 수행 
def activityModel():

    input_layer = Input(shape = (height, width, 1) ) # (4, 50, 1) = (feature, sliding_window, 1)

      # 50
    conv_0 = Conv2D(15, kernel_size=(1, 5), strides=(1, 1), padding='valid', activation='relu') (input_layer) #5
    conv_1 = Conv2D(15, kernel_size=(1, 5), strides=(1, 1), padding='same', activation='relu') (conv_0)
    dense_1 = Dense(50, activation='relu') (conv_1)
    pool_1 = MaxPooling2D(pool_size=(1,2)) (dense_1)
    drop_1 = Dropout(drop_prob_1)(pool_1)

      # 40
    conv_2 = Conv2D(11, kernel_size=(1, 3), strides=(1, 1), padding='valid', activation='relu') (drop_1)# 5
    dense_2 = Dense(40, activation='relu') (conv_2)
    pool_2 = MaxPooling2D(pool_size=(1, 3)) (dense_2)  
    drop_2 = Dropout(drop_prob_1) (pool_2)
      # 20
    conv_3 = Conv2D(10, kernel_size=(1, 3), strides=(1, 1), padding='same', activation='relu') (drop_2)
    drop_3 = Dropout(drop_prob_1)(conv_3)

    #------------------------------------ from now on...use drop_prob_2 
    flat = Flatten() (drop_3)
    hidden = Dense(400, activation = 'relu') (flat)
    drop_4 = Dropout(drop_prob_2)(hidden)

    #----------- Multi Task Output 
    action_out = Dense(action_layer_dim, activation = action_activation_func, name = 'ACTION') (drop_4)
    
    # --------- Model input/ Output Definition
    activity_model = Model(inputs=input_layer, outputs = action_out)
    activity_model.summary()

    activity_model.compile(loss=action_loss_func, 
          optimizer=keras.optimizers.Adam(0.001),
          metrics=metrics)

    return activity_model

In [6]:
## Training Phase
batch_size = 64
num_of_epochs = 5 #30
verbosity = 1

# Model Training 
model = activityModel()
history = model.fit(train_data, act_train_labels,                
              batch_size = batch_size,
              epochs = num_of_epochs,
              verbose = verbosity) 

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 4, 15, 1)]        0         
_________________________________________________________________
conv2d (Conv2D)              (None, 4, 11, 15)         90        
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 4, 11, 15)         1140      
_________________________________________________________________
dense (Dense)                (None, 4, 11, 50)         800       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 4, 5, 50)          0         
_________________________________________________________________
dropout (Dropout)            (None, 4, 5, 50)          0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 4, 3, 11)         

In [7]:
#------------- Evaluation 

results_1 = model.evaluate(test_data, act_test_labels,
                                 verbose = verbosity)

print("--> Evaluation on Test Dataset:")
print("**** Accuracy for Activity Recognition task is: ", results_1)

193/193 [==============================] - 0s 1ms/step - loss: 0.1766 - acc: 0.9063
--> Evaluation on Test Dataset:
**** Accuracy for Activity Recognition task is:  [0.17662864923477173, 0.9063463807106018]


In [14]:

y_predict = model.predict(test_data)

y_pred_int = np.round(y_predict,1)
print(y_predict.argmax(axis=1)) 
print(test_y[y_predict.argmax(axis=1)])
# print(y_predict.shape) # (6161, 5)  , sliding window size 

y_pred_max = y_predict.argmax(axis=1)
y_pred = y_pred_int[0][y_pred_max]
print(y_pred)
# x = np.arange(len(y_predict))

# fig, ax = plt.subplots(nrows=5, ncols=1, figsize=(20, 10))
# ax[0].plot(x, y_predict[:,0], 'g*', lw=0.1, label='sitting')
# ax[1].plot(x, y_predict[:,1], 'b*', lw=0.1, label='sitStand')
# ax[2].plot(x, y_predict[:,2], 'r*', lw=0.1, label='walking')
# ax[3].plot(x, y_predict[:,3], 'g*', lw=0.1, label='Turning')
# ax[4].plot(x, y_predict[:,4], 'b*', lw=0.1, label='standSit')
# plt.legend()
# plt.show()


[0 0 0 ... 4 4 4]
[[1. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0.]
 ...
 [1. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0.]]
(6161, 5)
[0.7 0.7 0.7 ... 0.  0.  0. ]


In [24]:
y_pred = test_y[y_predict.argmax(axis=1)]
y_test =np.argmax(test_y,axis=1)

correct = np.nonzero(y_pred == y_test)
incorrect = np.nonzero(y_pred != y_test)
print("correct: ", correct)

### Check the correctly-predicted samples
plt.figure(1)
for i, cor in enumerate(correct[:9]):
    plt.subplot(3,3,i+1)
    plt.subplots_adjust(hspace=1.0,wspace=1.0)
    plt.plot(test_x.iloc[cor,:])
    plt.title("Predicted:{}\nTrue:{}".format(activity_types[y_pred[cor]-1], 
                                              activity_types[y_test[cor]-1]))
    plt.xticks([]) # turn off x labels
    plt.yticks([])  # turn off y labels
    #plt.tight_layout()
plt.show()
### Check the incorrectly-predicted samples
plt.figure(2)
for i, cor in enumerate(incorrect[:9]):
    plt.subplot(3,3,i+1)
    plt.subplots_adjust(hspace=1.0,wspace=1.0)
    plt.plot(test_x.iloc[cor,:])
    plt.title("Predicted:{}\nTrue:{}".format(activity_types[y_pred[cor]-1], 
                                              activity_types[y_test[cor]-1]))
    plt.xticks([]) # turn off x labels
    plt.yticks([])  # turn off y labels
    #plt.tight_layout()
plt.show()

AttributeError: 'numpy.ndarray' object has no attribute 'iloc'

In [ ]:
### Confusion matrix (predictive performance on different classes)
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    function provided by sklearn example
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.tight_layout()

cnf_matrix = confusion_matrix(y_test, y_pred)
plot_confusion_matrix(cnf_matrix, classes=activity_types,
                      title='Confusion matrix, without normalization')